In [2]:
using Pkg
Pkg.activate(".")
using RemoteFiles
using Dash, PlotlyJS
using CSVFiles
using DataFrames
import TOML

function lookup(name)
           # Look up Registry.toml files in depots
           tomlfiles = String[]
           for d in DEPOT_PATH
               regs = joinpath(d, "registries")
               if isdir(regs)
                   for r in readdir(regs)
                       toml = joinpath(regs, r, "Registry.toml")
                       if isfile(toml)
                           push!(tomlfiles, toml)
                       end
                   end
               end
           end
           
           # Look up uuids in toml files
           uuids = Base.UUID[]
           for f in tomlfiles
               toml = TOML.parsefile(f)
               if haskey(toml, "packages")
                   for (k, v) in toml["packages"]
                       if v["name"] == name
                           push!(uuids, Base.UUID(k))
                       end
                   end
               end
           end
           return uuids
       end

data_dir = "../data/"
df = DataFrame(load(File(format"CSV", data_dir*"package_requests_by_date.csv.gz")));

  Activating environment at `~/Dropbox/Upwork/Plotly/JuliaPackages.jl/Project.toml`


In [3]:
uuid = string(lookup("Revise")[1])
mask = ((df."package_uuid" .== uuid) .& (df."status" .== 200) .& (df."client_type" .== "user"))
uuid_df = df[mask,:]
layout = Layout(;title="",width=800, height=400,
    xaxis=attr(title="date"), yaxis=attr(title="requests_count"))
fig = scatter(uuid_df, x=:date, y=:request_count, mode="markers+lines")
plot(fig, layout)

data: [
  "scatter with fields mode, type, x, and y"
]

layout: "layout with fields height, margin, template, title, width, xaxis, and yaxis"

In [ ]:
layout = Layout(;title="",width=600, height=600,
    xaxis=attr(title="date"), yaxis=attr(title="requests_count"))
fig = scatter(uuid_df, y=:date, mode="markers+lines")
plot(fig, layout)

In [6]:
df."date"[end]

2021-10-11

In [12]:
Set(uuid_df."client_type")

Set{String} with 1 element:
  "user"

In [7]:
lookup("Revise")

1-element Vector{Base.UUID}:
 UUID("295af30f-e4ad-537b-8983-00126c2a3abe")

In [15]:
Set(df."client_type")

Set{String} with 3 elements:
  "ci"
  "user"
  ""

In [18]:
a = []
append!(a,3)

1-element Vector{Any}:
 3

In [22]:
for q in Set([1,2,5])
    println(q)
end

5
2
1


In [20]:
df."client_type"

340107-element Vector{String}:
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 ⋮
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "ci"
 "user"
 "user"
 "user"
 "user"
 "user"

In [23]:
a = [
]

Any[]

In [24]:
append!(a,8)

1-element Vector{Any}:
 8

In [25]:
append!(a,9)

2-element Vector{Any}:
 8
 9

In [26]:
a

2-element Vector{Any}:
 8
 9

In [28]:
[fig]

1-element Vector{GenericTrace{Dict{Symbol, Any}}}:
 GenericTrace{Dict{Symbol, Any}}(Dict{Symbol, Any}(:mode => "markers+lines", :y => [268, 250, 247, 115, 137, 219, 216, 267, 272, 291  …  167, 355, 331, 421, 360, 342, 121, 96, 276, 322], :type => "scatter", :x => [Dates.Date("2021-09-01"), Dates.Date("2021-09-02"), Dates.Date("2021-09-03"), Dates.Date("2021-09-04"), Dates.Date("2021-09-05"), Dates.Date("2021-09-06"), Dates.Date("2021-09-07"), Dates.Date("2021-09-08"), Dates.Date("2021-09-09"), Dates.Date("2021-09-10")  …  Dates.Date("2021-10-03"), Dates.Date("2021-10-04"), Dates.Date("2021-10-05"), Dates.Date("2021-10-06"), Dates.Date("2021-10-07"), Dates.Date("2021-10-08"), Dates.Date("2021-10-09"), Dates.Date("2021-10-10"), Dates.Date("2021-10-11"), Dates.Date("2021-10-12")]))

In [37]:
sum(df[:,"request_count"])

66659008

In [32]:
setdiff(a,b)

Set{Int64} with 2 elements:
  2
  1

In [ ]:
"String"::String